# Capstone Project Notebook Model 1




In [2]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Copy of dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [6]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [7]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity", "SystemCodeNumber"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [8]:
#Model 1

df1= pd.read_csv("parking_stream.csv")
df1['Ratio']=df1['Occupancy']/df1['Capacity']

alpha_assume=0.005

df1['Price']= 0.0
df1.loc[0, 'Price']=10

for i in range(1,len(df1)):
  df1.loc[i, 'Price']=df1.loc[i-1, 'Price']+alpha_assume*df1.loc[i, 'Ratio']

df1['Price_diff']=df1['Price'].diff().fillna(0)

# Linear Regression to find Alpha

X=df1['Ratio'].values.reshape(-1,1)
y=df1['Price_diff'].values.reshape(-1,1)

alpha=np.linalg.inv(X.T@X)@X.T@y
alpha=alpha[0][0]

alpha=round(alpha,5)
print(alpha)

0.005


# Model 1 Pricing Function

In [9]:
df1['Price']=0.0
df1.loc[df.groupby('SystemCodeNumber').head(1).index, 'Price']=10

for code in df['SystemCodeNumber'].unique():
  current = df[df['SystemCodeNumber']==code]
  for i in range(1,len(current)):
    id=current.index[i]
    prev_id= current.index[i-1]
    df1.loc[id, 'Price']=df1.loc[prev_id, 'Price']+alpha*df1.loc[id,'Ratio']

df1['Occupancy']=df1['Occupancy'].astype(int)
df1['Capacity']=df1['Capacity'].astype(int)

df1[['Timestamp', 'Occupancy', 'Capacity', 'Price', 'SystemCodeNumber']].to_csv("parking_stream_1.csv",index=False)

In [10]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    Price: float       # Calculated price based on the occupancy and capacity
    SystemCodeNumber: str  # Unique identifier for the parking location

In [11]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream_1.csv", schema=ParkingSchema, input_rate=1000)

In [12]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_price = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    price= data.Price
)

In [13]:
print(list(data_with_price.keys()))

['Timestamp', 'Occupancy', 'Capacity', 'Price', 'SystemCodeNumber', 't', 'price']


In [14]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")

)



In [15]:
import datetime

# Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [16]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

In [17]:
pw.io.csv.write(data_with_price, "parking_plot.csv")

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [18]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

In [19]:
df_plot=pd.read_csv('parking_plot.csv')
print("Total Rows = ", len(df_plot))

Total Rows =  18368


In [20]:
df_plot["t"] = pd.to_datetime(df_plot["Timestamp"])

def park_plotter(df1):
  spots = df1["SystemCodeNumber"].unique()
  subplots=[]


  for spot in spots:
    subset = df1[df1["SystemCodeNumber"] == spot]

    fig=bokeh.plotting.figure(
        x_axis_type="datetime",
        height=400,
        width=800,
        title=f"Dynamic Parking Price for Spot {spot}"
    )

    fig.line(subset["t"], subset["price"], line_width=2, color="navy")
    fig.scatter(subset["t"], subset["price"], size=3, color="red")

    subplots.append(fig)

  return pn.Row(*subplots)


pn.panel(park_plotter(df_plot)).servable()

Row
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)

In [21]:
#For Comparison

df1.to_csv("model1_output.csv", index=False)